In [1]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

### Cargar Datos

In [2]:

with open('entrenamiento.p', 'rb') as file:
    training_data = pickle.load(file)
    

with open('prueba.p', 'rb') as file:
    test_data = pickle.load(file)
    

with open('validacion.p', 'rb') as file:
    validation_data = pickle.load(file)


### Dividir la data en X, Y. Entrenamiento, prueba y validación

In [3]:
X_train = training_data["features"]
y_train = training_data["labels"]

In [4]:
X_test = test_data["features"]
y_test = test_data["labels"]

In [5]:
X_val = validation_data["features"]
y_val = test_data["labels"]

### Normalizar los datos, debido a que los datos están con valores de 0 a 255 al dividirlos por 255 esto nos da un porcentaje de que tanto se acerca a 255.

In [6]:
X_train = X_train/255
X_test = X_test/255
X_val = X_val/255

In [7]:
y_train.shape

(34799,)

### Cargar los datos de entrenamiento y como son labels volverlos en categoricos desde 0 hasta 42, es decir 43 categorias diferentes

In [8]:
from tensorflow.keras.utils import to_categorical

In [9]:
y_cat_train = to_categorical(y_train, 43)


In [10]:
y_cat_test = to_categorical(y_test, 43)

In [11]:
y_cat_val = to_categorical(y_val, 43)

# Armado del modelo

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten

In [13]:
modelo = Sequential()

## Primer conjunto de capas

### Capa convolucional

Se crea una capa convulucional que tiene 32 filtros. Debido a que nuestras imagenesposeen un canal de colres, la forma del input sería de 32, 32, 3 siendo 3 el canal de los colores. Asimismo, el kernel sería de 4 por 4 lo cual quiere decir que se tomaran cuadrados de 4 por 4 pixeles para la capa de convolusión. Esto tendrá como output 32 imagenes diferentes, debido a los 32 filtros cada una de ellas con un tamaño de 29 (tamaño de imagen de entrada - (tamaño de kernel) + 1). 

In [15]:
modelo.add(Conv2D(filters = 32, kernel_size = (4, 4), input_shape = (32, 32, 3), activation = 'relu',))

### Capa de submuestreo (max-pooling)

Se crea una capa que realiza maxpooling. Este método lo que hace es crear una ventana del tamaño que se pida, en nuestro caso es de 2x2. En esa ventana de  4x4 se toma el valor del pixel más alto, es decir el valor maximo dentro de la ventana y luego se crea otra "imagen" de valores con ese nuevo valor. Esto se realiza con cada cuadro (2x2) de la imágen input ingresada. El output de esto es una imagen de 14 x 14. Esto se obtuvo a través de: floor((29-poolsize)/stride)+1. Ej. floor((29-2)/2) +1 => floor(27/2) +1 => floor(13.5)+1 => 13+1 = 14

In [16]:
modelo.add(MaxPool2D(pool_size = (2, 2)))

## Segundo conjunto de capas

Debido a que ahora se estan trabajando con colores. La cantidad de datos a procesar son demasiados por lo cual vale la pena realizar otra capa. Tanto convolucional como de submuestreo antes de analizar los datos a través de una red neuronal. 

### Capa convolucional 

In [18]:
modelo.add(Conv2D(filters = 32, kernel_size = (4, 4), input_shape = (32, 32, 3), activation = 'relu',))

### Capa de sub-muestreo (Pooling)

In [21]:
modelo.add(MaxPool2D(pool_size = (2, 2)))

### Flatten

In [23]:
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten

In [24]:
modelo.add(Flatten())

## 256 neuronas para la capa escondida con función de activación de Relu

In [26]:
modelo.add(Dense(256, activation = 'relu'))

### 43 outputs finales debido a que se tienen 43 categorías diferentes. Esta última capa tiene la función de decisión (activación) de Softmax

In [27]:
modelo.add(Dense(43, activation = 'softmax'))

# Compilación del modelo

### Para la compilación del modelo se utilizara la función de perdida de categorical crossentropy. Esta función sirve cuando se tiene una clasificación multi clase el cual es nuestro caso al tener 43 posibles decisiones.

In [30]:
modelo.compile(loss = 'categorical_crossentropy',
              optimizer = 'rmsprop',
              metrics = ['accuracy'])

In [31]:
modelo.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 29, 29, 32)        1568      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 32)        16416     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 32)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 800)               0         
                                                                 
 dense (Dense)               (None, 256)               2